In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import keras
from tensorflow.keras.utils import to_categorical
from keras import models
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.convolutional import Conv1D, MaxPooling1D
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/LeftWristDatasetWithlabels.csv')#,index_col='time_s',parse_dates=['time_s'])

In [ ]:
df = df.drop(['Unnamed: 0'], 1)
df = df.drop(['magnitude'], 1)

In [ ]:
df = df.sort_values(['subj_id', 'loc_id'], ascending=[True, True])
df

,subj_id,loc_id,time_s,x,y,z,label
180972,id00b70b13,left_wrist,0.00,-0.344,-1.285,-0.039,0.0
180973,id00b70b13,left_wrist,0.01,-0.336,-1.301,-0.027,0.0
180974,id00b70b13,left_wrist,0.02,-0.305,-1.324,-0.016,0.0
180975,id00b70b13,left_wrist,0.03,-0.273,-1.371,-0.004,0.0
180976,id00b70b13,left_wrist,0.04,-0.227,-1.418,0.020,0.0
...,...,...,...,...,...,...,...
647607,idff99de96,left_wrist,206.71,0.707,-0.570,0.289,0.0
647608,idff99de96,left_wrist,206.72,0.668,-0.555,0.254,0.0
647609,idff99de96,left_wrist,206.73,0.621,-0.527,0.207,0.0
647610,idff99de96,left_wrist,206.74,0.613,-0.488,0.168,0.0


In [ ]:
df['subj_id'] = df['subj_id'].astype('category')
df['subj_id'] = df['subj_id'].cat.codes

df['loc_id'] = df['loc_id'].astype('category')
df['loc_id'] = df['loc_id'].cat.codes

df['label'] = df['label'].astype('category')
df['label'] = df['label'].cat.codes

In [ ]:
df

,subj_id,loc_id,time_s,x,y,z,label
180972,0,0,0.00,-0.344,-1.285,-0.039,0
180973,0,0,0.01,-0.336,-1.301,-0.027,0
180974,0,0,0.02,-0.305,-1.324,-0.016,0
180975,0,0,0.03,-0.273,-1.371,-0.004,0
180976,0,0,0.04,-0.227,-1.418,0.020,0
...,...,...,...,...,...,...,...
647607,31,0,206.71,0.707,-0.570,0.289,0
647608,31,0,206.72,0.668,-0.555,0.254,0
647609,31,0,206.73,0.621,-0.527,0.207,0
647610,31,0,206.74,0.613,-0.488,0.168,0


# ***SCALING***

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# define min max scaler
scaler = MinMaxScaler()

# df1 = df[['loc_id','magnitude','label']]
# df1

In [ ]:
df_scaled = pd.DataFrame(scaler.fit_transform(df),columns=df.columns)

# df = pd.concat([df['subj_id'], df1], axis=1)
df = df_scaled

In [ ]:
df

,subj_id,loc_id,time_s,x,y,z,label
0,0.0,0.0,0.000000,0.406093,0.285967,0.354174,0.0
1,0.0,0.0,0.000041,0.407211,0.284198,0.356324,0.0
2,0.0,0.0,0.000083,0.411543,0.281654,0.358295,0.0
3,0.0,0.0,0.000124,0.416015,0.276457,0.360444,0.0
4,0.0,0.0,0.000166,0.422443,0.271260,0.364744,0.0
...,...,...,...,...,...,...,...
647607,1.0,0.0,0.855836,0.552963,0.365034,0.412934,0.0
647608,1.0,0.0,0.855877,0.547513,0.366692,0.406664,0.0
647609,1.0,0.0,0.855919,0.540945,0.369789,0.398244,0.0
647610,1.0,0.0,0.855960,0.539827,0.374102,0.391258,0.0


# **TRAINING AND TEST DATA**



*   Total data   : 32 Subjects
*   Training data: 25 subjects
*   Test data    : 7 subjects



In [ ]:
df.subj_id.unique()[:25]

array([0.        , 0.03225806, 0.06451613, 0.09677419, 0.12903226,
       0.16129032, 0.19354839, 0.22580645, 0.25806452, 0.29032258,
       0.32258065, 0.35483871, 0.38709677, 0.41935484, 0.4516129 ,
       0.48387097, 0.51612903, 0.5483871 , 0.58064516, 0.61290323,
       0.64516129, 0.67741935, 0.70967742, 0.74193548, 0.77419355])

In [ ]:
df_train = df[df.subj_id.isin( df.subj_id.unique()[:25] ) ] 

In [ ]:
df_test = df[df.subj_id.isin( df.subj_id.unique()[25:] )]

# ***Every subject was timed for 15minutes ***

# **overlapping windows with 50% overlap - Window size :50 Step size:50**

In [ ]:
from scipy import stats
x_list = []
y_list = []

train_labels = []
new_list = []

window_size = 50
step_size = 50

for i in range(0, df_train.shape[0] - window_size, step_size):
    xs = df_train['x'].values[i: i + 50]
    xs1 = df_train['y'].values[i: i + 50]
    xs2 = df_train['z'].values[i: i + 50]
    # ys = df_train['loc_id'].values[i: i + 50]
    # zs = df_train['subj_id'].values[i: i + 50]
    label = stats.mode(df_train['label'][i: i + 50])[0][0]
    a = np.c_[ xs,xs1,xs2 ]
    new_list.append(a)
    train_labels.append(label)

Xtrain = np.array(new_list)

In [ ]:
train_x = np.asarray(Xtrain)
train_y = np.asarray(train_labels)

# ***CNN***

In [ ]:
model_cnn = Sequential() 

model_cnn.add(Conv1D(filters=160, kernel_size=2, activation='relu', input_shape=(50,3)))

model_cnn.add(MaxPooling1D(pool_size=1))
model_cnn.add(Conv1D(filters=128, kernel_size=2, activation='relu'))
model_cnn.add(MaxPooling1D(pool_size=1))
model_cnn.add(Conv1D(filters=96, kernel_size=2, activation='relu'))
model_cnn.add(Conv1D(filters=64, kernel_size=2, activation='relu'))
model_cnn.add(MaxPooling1D(pool_size=1))
model_cnn.add(Flatten())
model_cnn.add(Dropout(0.5))
# model_cnn.add(Dense(64, activation='sigmoid'))
model_cnn.add(Dense(1,activation='sigmoid'))

model_cnn.compile(loss='binary_crossentropy', optimizer="adam", metrics = ["accuracy", tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])
model_cnn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_13 (Conv1D)          (None, 49, 160)           1120      
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 49, 160)          0         
 1D)                                                             
                                                                 
 conv1d_14 (Conv1D)          (None, 48, 128)           41088     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 48, 128)          0         
 1D)                                                             
                                                                 
 conv1d_15 (Conv1D)          (None, 47, 96)            24672     
                                                                 
 conv1d_16 (Conv1D)          (None, 46, 64)           

In [ ]:
model_cnn.fit(train_x, train_y, epochs=100, batch_size=1000, verbose=1)

Epoch 1/100
11/11 [==============================] - 2s 66ms/step - loss: 0.6932 - accuracy: 0.4949 - recall_4: 0.2405 - precision_4: 0.4780
Epoch 2/100
11/11 [==============================] - 1s 59ms/step - loss: 0.6935 - accuracy: 0.5033 - recall_4: 0.4409 - precision_4: 0.4971
Epoch 3/100
11/11 [==============================] - 1s 59ms/step - loss: 0.6933 - accuracy: 0.5051 - recall_4: 0.1266 - precision_4: 0.4969
Epoch 4/100
11/11 [==============================] - 1s 58ms/step - loss: 0.6930 - accuracy: 0.5061 - recall_4: 0.5230 - precision_4: 0.5002
Epoch 5/100
11/11 [==============================] - 1s 59ms/step - loss: 0.6936 - accuracy: 0.4981 - recall_4: 0.1526 - precision_4: 0.4754
Epoch 6/100
11/11 [==============================] - 1s 58ms/step - loss: 0.6932 - accuracy: 0.5009 - recall_4: 0.6924 - precision_4: 0.4964
Epoch 7/100
11/11 [==============================] - 1s 61ms/step - loss: 0.6931 - accuracy: 0.5052 - recall_4: 0.1372 - precision_4: 0.4975
Epoch 8/100
1

## ***Evaluation***

Test data

In [ ]:
from scipy import stats
test_y = []
test_x = []

window_size = 50
step_size = 50

for i in range(0, df_test.shape[0] - window_size, step_size):
    xs = df_test['x'].values[i: i + 50]
    xs1 = df_test['y'].values[i: i + 50]
    xs2 = df_test['z'].values[i: i + 50]
    # zs = df_train['subj_id'].values[i: i + 50]
    # ys = df_test['loc_id'].values[i: i + 50]
    label = stats.mode(df_test['label'][i: i + 50])[0][0]
    a = np.c_[ xs,xs1,xs2 ]
    test_x.append(a)
    test_y.append(label)

In [ ]:
test_x = np.array(test_x)
test_y = np.array(test_y)

scores = model_cnn.evaluate(test_x,test_y, verbose=1)

89/89 [==============================] - 1s 6ms/step - loss: 0.7021 - accuracy: 0.5025 - recall_4: 0.4935 - precision_4: 0.4968


# ***Prediction***

In [ ]:
pred = model_cnn.predict(test_x) 
pred = np.argmax(pred, axis = 1)
label = test_y

x = pd.DataFrame(pred,columns= ["preds"])


In [ ]:
test_y

array([0., 1., 1., ..., 0., 1., 0.])

# ***Confusion Matrix***

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_y, pred)

array([[1426,    0],
       [1394,    0]])

## **Classification Report**

In [ ]:
from sklearn import metrics 
from sklearn.metrics import classification_report
print("\n -------------Classification Report-------------\n")
print(classification_report(test_y, pred))


 -------------Classification Report-------------

              precision    recall  f1-score   support

         0.0       0.51      1.00      0.67      1426
         1.0       0.00      0.00      0.00      1394

    accuracy                           0.51      2820
   macro avg       0.25      0.50      0.34      2820
weighted avg       0.26      0.51      0.34      2820



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
